In [14]:

import numpy as np
import pandas as pd
import math


class BatchedDBSCAN:
    def __init__(
        self, z0, pt, eps, batch_size, max_number_of_tracks, verbose: bool = False
    ):

        self.eps = eps
        self.batch_size = batch_size
        self.verbose = verbose
        self.z0_boundary = 21  # 21 cm is outside the detector acceptance
        self.pt_boundary = 0  # 0 pT won't contribute to the pT sum.
        self.minPts = 2  # This algorithm only works for a minimum number of 2 points

        self.max_number_of_tracks = int(max_number_of_tracks)
        self.n_batches = math.ceil(self.max_number_of_tracks / self.batch_size)

        # Max number of tracks including all batches
        self.max_n_tracks_batched = self.batch_size * self.n_batches
        self.max_n_clusters_batch = math.ceil(self.batch_size / self.minPts)
        self.max_n_clusters = math.ceil(self.max_n_tracks_batched / self.minPts)

        # Need to pad vectors to the max_number_of_tracks allowed so that it matches the fpga input
        n_pad = self.max_number_of_tracks - z0.shape[0]
        self.z0 = self.pad_vector(z0, n_pad, self.z0_boundary)
        self.pt = self.pad_vector(pt, n_pad, self.pt_boundary)

        # These are needed for the prefix sum
        self.max_number_of_tracks_power_2 = (
            1 << (self.max_number_of_tracks - 1).bit_length()
        )
        self.batch_size_power_2 = 1 << (self.batch_size - 1).bit_length()
        self.max_number_of_tracks_log_2 = np.log2(self.max_number_of_tracks_power_2)
        self.batch_size_log_2 = np.log2(self.batch_size_power_2)
        # self.n_batches = math.ceil(self.max_number_of_tracks / self.batch_size)

    def pad_vector(self, vec, n_pad, value):
        """pads vector to a set size with given value"""

        vec_to_pad = value * np.ones(n_pad)
        vec = np.append(vec, vec_to_pad)

        return vec

    def build_tracks(self, z0, pt):
        """Builds tracks batchess"""

        # Shape is determined by the size of batch, z0, pT and label (not used atm)
        track_batch = np.zeros((self.batch_size, 3))

        track_batch[:, 0] = z0
        track_batch[:, 1] = pt

        # sort the tracks by z0
        track_batch = track_batch[track_batch[:, 0].argsort()]

        return track_batch

    def prefix_sum(self, arr):
        """
        Calculates the prefix sum of pT.
        Warning, requires array to be of size thats log base of 2.
        """
        size_log2 = int(np.log2(arr.shape[0]))

        # up-sweep
        for d in range(0, size_log2, 1):
            step_size = 2 ** d
            double_step_size = step_size * 2

            for i in range(0, arr.shape[0], double_step_size):
                arr[i + double_step_size - 1] += arr[i + step_size - 1]

        # down-sweep
        arr[arr.shape[0] - 1] = 0
        d = size_log2 - 1

        while d >= 0:
            step_size = 2 ** d
            double_step_size = step_size * 2
            for i in range(0, arr.shape[0], double_step_size):
                tmp = arr[i + step_size - 1]
                arr[i + step_size - 1] = arr[i + double_step_size - 1]
                arr[i + double_step_size - 1] += tmp
            d -= 1

        return arr

    def find_left_boundaries(self, tracks):

        left_boundaries = np.zeros(self.batch_size, dtype=bool)

        # first value is always a left boundary
        left_boundaries[0] = 1

        for i in range(1, self.batch_size):
            _t = tracks[i]

            if _t[0] - tracks[i - 1][0] > self.eps:
                tracks[i][2] = -1
                left_boundaries[i] = 1
            else:
                left_boundaries[i] = 0

        return left_boundaries

    def find_right_boundaries(self, left_boundaries, rs, tracks):

        max_tracks = self.batch_size

        boundaries = np.zeros((max_tracks, 6))

        for i in range(max_tracks - 1):

            check1 = left_boundaries[i] and not (left_boundaries[i + 1])
            check2 = not (left_boundaries[i]) and left_boundaries[i + 1]

            if check1 or check2:
                boundaries[i][0] = i
                boundaries[i][1] = rs[i]
                boundaries[i][2] = rs[i + 1]
                boundaries[i][3] = rs[i + 1] - rs[i]
                boundaries[i][4] = tracks[i, 0]
                boundaries[i][5] = tracks[i + 1, 0]
            else:
                boundaries[i][0] = max_tracks
                boundaries[i][1] = 0
                boundaries[i][2] = 0
                boundaries[i][3] = 0
                boundaries[i][4] = 21
                boundaries[i][5] = 21

        # Check for the last boundary
        if left_boundaries[max_tracks - 1]:
            boundaries[max_tracks - 1][0] = max_tracks
            boundaries[max_tracks - 1][1] = 0
            boundaries[max_tracks - 1][2] = 0
            boundaries[max_tracks - 1][3] = 0
            boundaries[max_tracks - 1][4] = 21
            boundaries[max_tracks - 1][5] = 21
        else:
            boundaries[max_tracks - 1][0] = max_tracks - 1
            boundaries[max_tracks - 1][1] = rs[max_tracks - 1]
            boundaries[max_tracks - 1][2] = rs[max_tracks]

        # Sort boundaries by the index
        boundaries = boundaries[boundaries[:, 0].argsort()]

        # # Add sum of Pt information
        # boundaries[:, 3] = boundaries[:, 2] - boundaries[:,  1]

        return boundaries

    def convert_boundaries_to_clusters(self, boundaries):

        clusters = np.zeros((self.max_n_clusters_batch, 6))
        for i in range(boundaries.shape[0]):
            if boundaries[i, -1] >= 21:
                break
            clusters[i, :] = boundaries[i, :]
        return clusters

    def fit(self):

        np.set_printoptions(precision=2)
        np.set_printoptions(suppress=True)

        start_idx = 0
        end_idx = start_idx + self.batch_size

        # Need to pad vectors to match the size of n_batches*batch_size
        n_pad = (self.n_batches * self.batch_size) - self.z0.shape[0]
        self.z0 = self.pad_vector(self.z0, n_pad, 21)
        self.pt = self.pad_vector(self.pt, n_pad, 0)

        max_rs = 0

        clusters = np.zeros((self.max_number_of_tracks, 6))
        print(clusters.shape)
        # final_clusters = np.zeros((self.n_batches * self.batch_size, 6))
        # final_clusters = np.zeros((self.max_n_clusters, 6))
        pv_cluster = np.zeros((1, 6))

        for i in range(self.n_batches):
            if i == 1:
                break
            print(f"----------------- {i} --------------")
            start_idx = i * self.batch_size
            end_idx = (i + 1) * self.batch_size

            z0_batch = self.z0[start_idx:end_idx]
            pt_batch = self.pt[start_idx:end_idx]
            # print(z0_batch.shape, pt_batch.shape)

            track_batch = self.build_tracks(z0_batch, pt_batch)

            rs_batch = self.pad_vector(
                track_batch[:, 1], self.batch_size_power_2 - self.batch_size, 0
            )
            # print(rs_batch.shape)
            # print(rs_batch)
            # print(track_batch[:, 0])
            # print(track_batch[:, 1])
            # print(rs_batch)
            rs_batch = self.prefix_sum(rs_batch)
            self.rs_batch = rs_batch
            # print(rs_batch)
            # rs_batch += max_rs
            # max_rs = rs_batch[-1]

            left_boundaries = self.find_left_boundaries(track_batch)

            boundaries = self.find_right_boundaries(
                left_boundaries, rs_batch, track_batch
            )

            clusters_batch = self.convert_boundaries_to_clusters(boundaries)
            self.clusters_batch = clusters_batch
            self.tracks_batch = track_batch
            print(clusters_batch)
            # print(clusters_batch.shape)
            # print(clusters.shape)
            print(f"[{i*self.max_n_clusters_batch}, {(i+1)*self.max_n_clusters_batch}]")
            clusters[
                i * self.max_n_clusters_batch : (i + 1) * self.max_n_clusters_batch, :
            ] = clusters_batch

            if track_batch[-1, 0] == 21:
                break

        # Merge clusters
        max_pt = 0
        max_pt_i = 0
        merge_count = 0
        run_count = 0
        for i in range(clusters.shape[0]):
            if max_pt < clusters[i, 3]:
                max_pt = clusters[i, 3]
                max_pt_i = i
            for j in range(clusters.shape[0]):

                if i >= j:
                    continue

                case1 = (clusters[j, 4] < clusters[i, 5] + self.eps) and (
                    clusters[j, 5] < clusters[i, 4] - self.eps
                )
                case2 = (clusters[j, 5] > clusters[i, 4] - self.eps) and (
                    clusters[j, 4] < clusters[i, 5] + self.eps
                )
                run_count +=1
                if case1 or case2:
                    # print("merging")
                    merge_count += 1
                    if clusters[j, 4] < clusters[i, 4]:
                        clusters[i, 4] = clusters[j, 4]
                    if clusters[j, 5] > clusters[i, 5]:
                        clusters[i, 5] = clusters[j, 5]
                    clusters[i, 3] += clusters[j, 3]
                    if max_pt < clusters[i, 3]:
                        max_pt = clusters[i, 3]
                        max_pt_i = i

        # Find pv_cluster
        pv_cluster[0, :] = clusters[max_pt_i, :]
        # np.save("clusters.npy", clusters)
        print(max_pt, max_pt_i)
        print(pv_cluster)
        print(f"Merged count: {merge_count} / {run_count} ({100*(merge_count/run_count)})")

        pv_tracks = []

        for i in range(self.max_number_of_tracks):
            z0_trk = self.z0[i]

            if (z0_trk >= pv_cluster[0, 4]) and (z0_trk <= pv_cluster[0, 5]):
                pv_tracks.append(z0_trk)

        print(f"mean: {np.mean(pv_tracks)}")
        print(f"median: {np.median(pv_tracks)}")


In [15]:
import numpy as np

max_number_of_tracks = 232
max_number_of_tracks_power_2 = 256
max_number_of_tracks_log_2 = 8
batch_size = 50
eps = 0.15

z0_file = "/media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-z0.bin"
pt_file = "/media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-pt.bin"
z0 = np.fromfile(z0_file, dtype=np.float32)
pt = np.fromfile(pt_file, dtype=np.float32)

db = BatchedDBSCAN(z0, pt, eps, batch_size, max_number_of_tracks, True)

db.fit()



(232, 6)
----------------- 0 --------------
[[  0.     0.     4.83   4.83  -5.62  -5.51]
 [  1.     4.83   7.96   3.13  -5.51  -5.1 ]
 [  3.    10.66  13.55   2.89  -4.92  -4.92]
 [  4.    13.55  15.51   1.96  -4.92  -4.69]
 [  7.    19.82  22.03   2.21  -4.04  -4.04]
 [  8.    22.03  24.15   2.12  -4.04  -3.69]
 [  9.    24.15  26.12   1.98  -3.69  -3.69]
 [ 13.    42.14  49.23   7.1   -3.52  -3.05]
 [ 14.    49.23  51.34   2.11  -3.05  -2.93]
 [ 16.    53.77  55.75   1.97  -2.93  -2.58]
 [ 18.    57.84  60.63   2.8   -2.4   -2.34]
 [ 26.    97.91 100.37   2.46  -1.88  -1.7 ]
 [ 28.   103.08 107.52   4.44  -1.46  -1.41]
 [ 29.   107.52 109.98   2.46  -1.41  -0.94]
 [ 32.   114.97 119.07   4.1   -0.41  -0.29]
 [ 33.   119.07 121.22   2.15  -0.29   0.06]
 [ 35.   123.19 126.87   3.67   0.23   0.23]
 [ 38.   131.38 133.8    2.42   0.41   0.88]
 [ 47.   159.78 161.86   2.08   6.5    6.62]
 [ 48.   161.86 163.94   2.08   6.62   7.03]
 [  0.     0.     0.     0.     0.     0.  ]
 [  0.     

In [16]:
db.clusters_batch

array([[  0.  ,   0.  ,   4.83,   4.83,  -5.62,  -5.51],
       [  1.  ,   4.83,   7.96,   3.13,  -5.51,  -5.1 ],
       [  3.  ,  10.66,  13.55,   2.89,  -4.92,  -4.92],
       [  4.  ,  13.55,  15.51,   1.96,  -4.92,  -4.69],
       [  7.  ,  19.82,  22.03,   2.21,  -4.04,  -4.04],
       [  8.  ,  22.03,  24.15,   2.12,  -4.04,  -3.69],
       [  9.  ,  24.15,  26.12,   1.98,  -3.69,  -3.69],
       [ 13.  ,  42.14,  49.23,   7.1 ,  -3.52,  -3.05],
       [ 14.  ,  49.23,  51.34,   2.11,  -3.05,  -2.93],
       [ 16.  ,  53.77,  55.75,   1.97,  -2.93,  -2.58],
       [ 18.  ,  57.84,  60.63,   2.8 ,  -2.4 ,  -2.34],
       [ 26.  ,  97.91, 100.37,   2.46,  -1.88,  -1.7 ],
       [ 28.  , 103.08, 107.52,   4.44,  -1.46,  -1.41],
       [ 29.  , 107.52, 109.98,   2.46,  -1.41,  -0.94],
       [ 32.  , 114.97, 119.07,   4.1 ,  -0.41,  -0.29],
       [ 33.  , 119.07, 121.22,   2.15,  -0.29,   0.06],
       [ 35.  , 123.19, 126.87,   3.67,   0.23,   0.23],
       [ 38.  , 131.38, 133.8 ,

In [17]:
columns = ["idx", "pt_min", "pt_max", "pt_sum", "z0_min", "z0_max"]

In [18]:
df = pd.DataFrame(db.clusters_batch, columns = columns)

In [13]:
df

,idx,pt_min,pt_max,pt_sum,z0_min,z0_max
0,0.0,0.000000,4.826959,4.826959,-5.625000,-5.507812
1,1.0,4.826959,7.957812,3.130853,-5.507812,-5.097656
2,3.0,10.656403,13.549811,2.893408,-4.921875,-4.921875
3,4.0,13.549811,15.508527,1.958716,-4.921875,-4.687500
4,7.0,19.816681,22.027795,2.211114,-4.042969,-4.042969
5,8.0,22.027795,24.147307,2.119512,-4.042969,-3.691406
6,9.0,24.147307,26.124864,1.977557,-3.691406,-3.691406
7,13.0,42.135565,49.234034,7.098469,-3.515625,-3.046875
8,14.0,49.234034,51.341876,2.107842,-3.046875,-2.929688
9,16.0,53.774925,55.749668,1.974743,-2.929688,-2.578125


In [20]:
db.tracks_batch.shape

(50, 3)

In [24]:
db.tracks_batch[:,0]

array([-5.62, -5.51, -5.1 , -4.92, -4.92, -4.69, -4.22, -4.04, -4.04,
       -3.69, -3.69, -3.63, -3.57, -3.52, -3.05, -2.93, -2.93, -2.58,
       -2.4 , -2.34, -2.23, -2.23, -2.23, -2.17, -2.05, -1.99, -1.88,
       -1.7 , -1.46, -1.41, -0.94, -0.76, -0.41, -0.29,  0.06,  0.23,
        0.23,  0.29,  0.41,  0.88,  1.35,  2.11,  2.81,  3.57,  3.87,
        5.1 ,  5.74,  6.5 ,  6.62,  7.03])

In [25]:
z0 = pd.DataFrame(db.tracks_batch[:,0], columns=['z0'])

In [28]:
z0['diff'] = z0.diff()

In [29]:
z0

,z0,diff
0,-5.625000,NaN
1,-5.507812,0.117188
2,-5.097656,0.410156
3,-4.921875,0.175781
4,-4.921875,0.000000
5,-4.687500,0.234375
6,-4.218750,0.468750
7,-4.042969,0.175781
8,-4.042969,0.000000
9,-3.691406,0.351562


In [30]:
db.clusters_batch

array([[  0.  ,   0.  ,   4.83,   4.83,  -5.62,  -5.51],
       [  1.  ,   4.83,   7.96,   3.13,  -5.51,  -5.1 ],
       [  3.  ,  10.66,  13.55,   2.89,  -4.92,  -4.92],
       [  4.  ,  13.55,  15.51,   1.96,  -4.92,  -4.69],
       [  7.  ,  19.82,  22.03,   2.21,  -4.04,  -4.04],
       [  8.  ,  22.03,  24.15,   2.12,  -4.04,  -3.69],
       [  9.  ,  24.15,  26.12,   1.98,  -3.69,  -3.69],
       [ 13.  ,  42.14,  49.23,   7.1 ,  -3.52,  -3.05],
       [ 14.  ,  49.23,  51.34,   2.11,  -3.05,  -2.93],
       [ 16.  ,  53.77,  55.75,   1.97,  -2.93,  -2.58],
       [ 18.  ,  57.84,  60.63,   2.8 ,  -2.4 ,  -2.34],
       [ 26.  ,  97.91, 100.37,   2.46,  -1.88,  -1.7 ],
       [ 28.  , 103.08, 107.52,   4.44,  -1.46,  -1.41],
       [ 29.  , 107.52, 109.98,   2.46,  -1.41,  -0.94],
       [ 32.  , 114.97, 119.07,   4.1 ,  -0.41,  -0.29],
       [ 33.  , 119.07, 121.22,   2.15,  -0.29,   0.06],
       [ 35.  , 123.19, 126.87,   3.67,   0.23,   0.23],
       [ 38.  , 131.38, 133.8 ,